In [2]:
import struct

# =========================
# Helper: Read VarInt
# =========================
def read_varint(data, offset):
    prefix = data[offset]

    if prefix < 0xfd:
        return prefix, offset + 1

    elif prefix == 0xfd:
        value = struct.unpack('<H', data[offset+1:offset+3])[0]
        return value, offset + 3

    elif prefix == 0xfe:
        value = struct.unpack('<I', data[offset+1:offset+5])[0]
        return value, offset + 5

    else:
        value = struct.unpack('<Q', data[offset+1:offset+9])[0]
        return value, offset + 9


# =========================
# Main Parser
# =========================
def parse_transaction(raw_hex):

    data = bytes.fromhex(raw_hex)
    offset = 0

    # 1️⃣ Version (4 bytes)
    version = struct.unpack('<I', data[offset:offset+4])[0]
    offset += 4

    # 2️⃣ Input Count (VarInt)
    input_count, offset = read_varint(data, offset)

    inputs = []

    for _ in range(input_count):

        # Previous TXID (32 bytes, little-endian)
        prev_txid = data[offset:offset+32][::-1].hex()
        offset += 32

        # Output index (4 bytes)
        vout = struct.unpack('<I', data[offset:offset+4])[0]
        offset += 4

        # Script length
        script_length, offset = read_varint(data, offset)

        # ScriptSig
        script_sig = data[offset:offset+script_length].hex()
        offset += script_length

        # Sequence
        sequence = data[offset:offset+4].hex()
        offset += 4

        inputs.append({
            "prev_txid": prev_txid,
            "vout": vout,
            "script_sig": script_sig,
            "sequence": sequence
        })

    # 3️⃣ Output Count (VarInt)
    output_count, offset = read_varint(data, offset)

    outputs = []

    for _ in range(output_count):

        # Value (8 bytes)
        value_satoshi = struct.unpack('<Q', data[offset:offset+8])[0]
        offset += 8

        value_btc = value_satoshi / 1e8

        # Script length
        script_length, offset = read_varint(data, offset)

        # ScriptPubKey
        script_pubkey = data[offset:offset+script_length].hex()
        offset += script_length

        outputs.append({
            "value_satoshi": value_satoshi,
            "value_btc": value_btc,
            "script_pubkey": script_pubkey
        })

    # 4️⃣ Locktime (4 bytes)
    locktime = struct.unpack('<I', data[offset:offset+4])[0]

    return {
        "version": version,
        "input_count": input_count,
        "inputs": inputs,
        "output_count": output_count,
        "outputs": outputs,
        "locktime": locktime
    }


# =========================
# Example Transaction
# =========================

raw_tx = "01000000018b0f9e4c5c6f0e6f8d0b2a3b4c5d6e7f8a9b0c1d2e3f4a5b6c7d8e9f0a1b2c3d000000006b483045022100f3b3c3a7d0f1e2d3c4b5a697887766554433221100ffeeddccbbaa9988776655022034ab56cd78ef90ab12cd34ef56ab78cd90ef12ab34cd56ef78ab90cd12ef34ab0121027d1c4e5f6a7b8c9d0e1f2233445566778899aabbccddeeff0011223344556677ffffffff01a0860100000000001976a91489abcdefabbaabbaabbaabbaabbaabbaabbaabba88ac00000000"

parsed = parse_transaction(raw_tx)

import pprint
pprint.pprint(parsed)

{'input_count': 1,
 'inputs': [{'prev_txid': '3d2c1b0a9f8e7d6c5b4a3f2e1d0c9b8a7f6e5d4c3b2a0b8d6f0e6f5c4c9e0f8b',
             'script_sig': '483045022100f3b3c3a7d0f1e2d3c4b5a697887766554433221100ffeeddccbbaa9988776655022034ab56cd78ef90ab12cd34ef56ab78cd90ef12ab34cd56ef78ab90cd12ef34ab0121027d1c4e5f6a7b8c9d0e1f2233445566778899aabbccddeeff0011223344556677',
             'sequence': 'ffffffff',
             'vout': 0}],
 'locktime': 0,
 'output_count': 1,
 'outputs': [{'script_pubkey': '76a91489abcdefabbaabbaabbaabbaabbaabbaabbaabba88ac',
              'value_btc': 0.001,
              'value_satoshi': 100000}],
 'version': 1}
